In [1]:
import pandas as pd
from pathlib import Path

In [2]:
model = 'gemma2-2b'
dist_folder = 'no-distil-test'
filename = 'FINAL_Completed(0 - 1319).csv'

In [3]:
basepath = (Path() / '..' /'..'/ 'logs' / model / dist_folder).resolve()

In [4]:
df = pd.read_csv(basepath / filename)

In [7]:
ex_df = pd.read_csv((Path() / '..' / '..' / 'logs' / 'gemma2b-it' / dist_folder / 'FINAL_RESULTS.csv').resolve())

In [9]:
df['extracted_answer'] = ex_df['extracted_answer']

In [10]:
for i, row in df.iterrows():
    df.loc[i, 'model_answer'] = row['model_answer'].removeprefix(row['question'])
    df.loc[i, 'model_answer'] = df.loc[i, 'model_answer'].removeprefix('\n Please respond with just the answer. The answer is: ')
    # df.loc[i, 'extracted_answer'] = row['answer'].split('####')[1].strip()

In [59]:
import re
df['model_ex1'] = df['model_answer']
for i, row in df.iterrows():
    extr = re.findall(r'[-+]?[$]?([,\d]+\.?[,\d]*)\%?(\ |$|\n){1}', row['model_answer'])
    if len(extr) != 1:  
        df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])
        continue
    v = re.sub(r',', '', extr[0][0])
    try:
        df.loc[i, 'model_ex1'] = float(v)
    except:
        df.loc[i, 'model_ex1'] = v
    df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])

In [60]:
df['model_ex1_t'].value_counts()

model_ex1_t
<class 'float'>    1218
<class 'str'>       101
Name: count, dtype: int64

In [61]:
def save_df(df : pd.DataFrame, model : str, dist_folder : str, edition : str):
    base_path = Path()
    base_path = base_path / '..' /'..'/ 'logs' / model / dist_folder / f'{edition}.csv'
    df.to_csv(base_path)

In [63]:
df_false = df[df['model_ex1_t'] == str]
df_false['model_ex1'].head(10)

In [78]:

for i, row in df[df['model_ex1_t'] != float].iterrows():
    inp = input(f'{i} : What is the extracted answer : {row["model_ex1"]}')
    try:
        df.loc[i, 'model_ex1'] = float(inp)
    except:
        print(f'{i} could not be parsed')

212 could not be parsed
509 could not be parsed
524 could not be parsed
753 could not be parsed
906 could not be parsed
1017 could not be parsed
1153 could not be parsed


In [80]:
for i, row in df.iterrows():
    df.loc[i, 'model_ex1_t'] = type(row['model_ex1'])


In [ ]:
for i, row in df[df['model_ex1_t'] != float].iterrows():
    print(f'Model Answer: {row['model_ex1']}')
    print(f'True Answer: {row['answer']}')
    print(f'Question: {row['question']}')
    print()
    inp = input(f'{i} : What is the extracted answer : {row["model_ex1"]}')
    try:
        df.loc[i, 'model_ex1'] = float(inp)
    except:
        print(f'{i} could not be parsed')
    df.loc[i, 'model_ex1_t'] = type(row['model_ex1'])
    

In [85]:
df['correct'] = df['model_ex1'] == df['extracted_answer']

In [86]:
df['correct'].value_counts()

correct
False    1248
True       71
Name: count, dtype: int64

In [88]:
save_df(df, 'gemma2-2b', 'no-distil-test', 'FINAL_RESULTS')